### Pitchfork Content Sandbox

#### Vectorization
this section turns a collection of Pitchfork music reviews into numerical feature vectors, including tokenization, counting and normalization. Bag of Words-tf-idf representation where reviews are described by word occurrences while completely ignoring the relative position information of the words in the document.

In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import psycopg2
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from statsmodels.stats.contingency_tables import mcnemar

import numpy as np
np.set_printoptions(threshold=np.inf)
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from pitchfork_content_functions import vectorize

pd.option_context('display.max_colwidth', -1)
pd.options.display.max_rows = 1000
pd.options.display.max_seq_items = 5000

In [2]:
# create connection
conn = psycopg2.connect("dbname=pitchfork_reviews")
cur = conn.cursor()

# query
cur.execute("""
SELECT genres.genre, content.reviewid, content.content 
FROM content
INNER JOIN genres on content.reviewid = genres.reviewid;
""")

# cast to dataframe
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]

In [3]:
df.head(), len(df), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22690 entries, 0 to 22689
Data columns (total 3 columns):
genre       20319 non-null object
reviewid    22690 non-null int64
content     22690 non-null object
dtypes: int64(1), object(2)
memory usage: 531.9+ KB


(        genre  reviewid                                            content
 0  electronic     22703  “Trip-hop” eventually became a ’90s punchline,...
 1       metal     22721  Eight years, five albums, and two EPs in, the ...
 2        rock     22659  Minneapolis’ Uranium Club seem to revel in bei...
 3        rock     22661  Kleenex began with a crash. It transpired one ...
 4  electronic     22725  It is impossible to consider a given release b...,
 22690,
 None)

In [4]:
# dropping ~20K nulls in genre columns
df = df.dropna(how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20319 entries, 0 to 22688
Data columns (total 3 columns):
genre       20319 non-null object
reviewid    20319 non-null int64
content     20319 non-null object
dtypes: int64(1), object(2)
memory usage: 635.0+ KB


In [5]:
data = df['content'].astype(str)
data.head(5)

0    “Trip-hop” eventually became a ’90s punchline,...
1    Eight years, five albums, and two EPs in, the ...
2    Minneapolis’ Uranium Club seem to revel in bei...
3    Kleenex began with a crash. It transpired one ...
4    It is impossible to consider a given release b...
Name: content, dtype: object

In [6]:
df_genre = pd.DataFrame(df['genre'])
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20319 entries, 0 to 22688
Data columns (total 1 columns):
genre    20319 non-null object
dtypes: object(1)
memory usage: 317.5+ KB


In [7]:
df['genre'].value_counts()

rock            9438
electronic      3874
experimental    1815
rap             1559
pop/r&b         1432
metal            862
folk/country     685
jazz             435
global           219
Name: genre, dtype: int64

In [8]:
feature_names = df['genre'].astype(str)
feature_names[:5]

0    electronic
1         metal
2          rock
3          rock
4    electronic
Name: genre, dtype: object

In [9]:
label_encoder = LabelEncoder()
feature_names_arr = label_encoder.fit_transform(feature_names)
feature_names_arr.shape

(20319,)

In [10]:
len(feature_names), type(feature_names)

(20319, pandas.core.series.Series)

In [11]:
np.vstack((feature_names_arr[:10], feature_names[:10]))

array([[0, 5, 8, 8, 0, 7, 0, 1, 7, 1],
       ['electronic', 'metal', 'rock', 'rock', 'electronic', 'rap',
        'electronic', 'experimental', 'rap', 'experimental']],
      dtype=object)

In [12]:
# nine classes
label_encoder.classes_, len(label_encoder.classes_)

(array(['electronic', 'experimental', 'folk/country', 'global', 'jazz',
        'metal', 'pop/r&b', 'rap', 'rock'], dtype=object), 9)

In [13]:
len(data), len(feature_names_arr)

(20319, 20319)

In [14]:
feature_names_arr[0], data[0]

(0,
 "“Trip-hop” eventually became a ’90s punchline, a music-press shorthand for “overhyped hotel lounge music.” But today, the much-maligned subgenre almost feels like a secret precedent. Listen to any of the canonical Bristol-scene albums of the mid-late ’90s, when the genre was starting to chafe against its boundaries, and you’d think the claustrophobic, anxious 21st century started a few years ahead of schedule. Looked at from the right angle, trip-hop\xa0is part of an unbroken chain that runs from the abrasion of ’80s post-punk to the ruminative pop-R&B-dance fusion of the moment.\xa0The best of it has aged far more gracefully (and forcefully) than anything recorded in the waning days of the record industry’s pre-filesharing monomania has any right to. Tricky rebelled against being attached at the hip to a scene he was already looking to shed and decamped for Jamaica to record a more aggressive, bristling-energy mutation of his style in ’96; the name\xa0Pre-Millennium Tension is t

#### Partition Data

In [15]:
# splits data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(data, feature_names_arr, test_size=0.30, random_state=3)

In [16]:
len(X_train), len(X_test), len(data) * .7

(14223, 6096, 14223.3)

In [17]:
len(X_train), len(X_test), len(y_train), len(y_test)

(14223, 6096, 14223, 6096)

#### Vectorize data

In [18]:
# def vectorize(corpus):
#    """learns vocab dictionary and returns feature names and term-document matrix"""
    
# count_vect = CountVectorizer()X_train_counts = count_vect.fit_transform(twenty_train.data)
  #  X_train_counts.shape

In [19]:
# count_vectorizer = CountVectorizer(lowercase=True)
# X_train_count = count_vectorizer.fit_transform(X_train)
# vectorizer.get_feature_names(), X.toarray()

In [20]:
# returns sparse matrices for every document
count_vect = CountVectorizer(max_features=1000, stop_words='english')
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(14223, 1000)

In [21]:
X_train_counts[0]

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 84 stored elements in Compressed Sparse Row format>

In [22]:
for i in range(1000):
    print(X_train_counts[0,i])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
0
0
0
0
1
0
0
0
0
0
2
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [23]:
X_train_counts[1]

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 120 stored elements in Compressed Sparse Row format>

In [24]:
X_train_counts[1]

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 120 stored elements in Compressed Sparse Row format>

In [25]:
count_vect.vocabulary_

{'common': 159,
 'electro': 257,
 'pop': 652,
 'artists': 51,
 'ambient': 36,
 'project': 675,
 'work': 980,
 'away': 58,
 'makes': 526,
 'sense': 771,
 'music': 573,
 'vibe': 944,
 'feel': 301,
 'songs': 810,
 'way': 963,
 'atmosphere': 53,
 'lot': 514,
 'natural': 580,
 'approach': 45,
 'certain': 131,
 'going': 362,
 'just': 454,
 'break': 101,
 'songwriting': 812,
 'album': 33,
 'non': 590,
 'musical': 574,
 'elements': 260,
 'make': 525,
 'fact': 287,
 'recorded': 711,
 'days': 197,
 'past': 622,
 'record': 710,
 'half': 383,
 'electric': 256,
 'guitar': 377,
 'instrument': 442,
 'didn': 210,
 'heavily': 400,
 'debut': 201,
 'lp': 518,
 'year': 994,
 'recent': 708,
 'piano': 636,
 'ear': 245,
 'material': 531,
 'offers': 602,
 'form': 330,
 'playing': 645,
 'patterns': 624,
 'static': 838,
 'tone': 903,
 'bit': 88,
 'particularly': 619,
 'quite': 693,
 'hear': 396,
 'notes': 593,
 'opener': 607,
 'real': 704,
 'movement': 568,
 'highlight': 406,
 'ultimately': 927,
 'feels': 303,


In [26]:
# after fitting, the vectorizer, "count_vect" has built a dictionary of feature indices
# The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.
len(count_vect.vocabulary_)

1000

#### Term frequencies (TF) and  term frequency inverse document frequency (TF-IDF) using `TfidfTransformer`
- TF divides the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

- TF-IDF downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

#### tf - TODO (Lee) - I believe this 'tf' section made redundant by alternate sklearn function by Miles

In [61]:
# TODO (Lee) - correct use_idf true here, I think, if tf
# computes TF-IDF using TfidfTransformer
# tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts) # fits/learns idf vector (global term weights)

In [62]:
# tf_transformer

In [63]:
# X_train_tf = tf_transformer.transform(X_train_counts) # transform count matrix to a tf representation

In [64]:
# TODO (Lee) (I think) X_train_tf[i,j] is the occurrence of word j in the text i.

# X_train_tf[0,0]

In [65]:
# TODO (Lee) - complete this comment to clarify what we are seeing here
# X_train_tf.sum(axis=0).shape, X_train_tf.sum(axis=0)

In [66]:
# TODO (Lee) - I'm not sure what this did, why it is 35788
# X_train_tf.shape
(2257, 35788)

(2257, 35788)

#### tf-idf

In [33]:
# computes TF-IDF using TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts) # fits/learns idf vector (global term weights)

In [34]:
tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [35]:
X_train_tfidf = tf_transformer.transform(X_train_counts) # transform count matrix to a tf-idf representation

In [36]:
X_train_tfidf.shape, len(y_train)

((14223, 1000), 14223)

### Train naive bayes classifier to predict genres on test music reviews

In [37]:
# train Naive Bayes Classifier on training features (X_train_tfidf) and training targets (y_train)

# model = MultinomialNB()
# model.fit(X_train_tf, y_train)

In [38]:
# vectorizes X_test set, similar to above for train set EXCEPT call transform, NOT fit_transform, since fit on train set
# since they have already been fit to the training set:

# X_test_counts = count_vect.transform(X_test)

In [39]:
# X_test_counts

In [40]:
# X_test_tfidf = tf_transformer.transform(X_test_counts)

In [67]:
tf_vect = TfidfVectorizer()
tf_vect.fit(X_train)
X_train_tf = tf_vect.transform(X_train)

In [48]:
# TODO (Lee) - note here the X_train_tf convention that I think was from a Miles implementation
model = MultinomialNB()
model.fit(X_train_tf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [68]:
X_test_tfidf = tf_vect.transform(X_test)

In [ ]:
# predicted = model.predict(X_test_tfidf)
# X_new_counts = count_vect.transform(X_test)
# X_new_tfidf = TfidfTransformer.transform(X_test)

In [50]:
# when calling len() on sparse matrix - TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]
# inspect shapes - .shape[0] is for sparse matrices, and getnnz gets the count of explicitly-stored values (nonzeros)
X_test_tfidf.shape[0], X_test_tfidf.getnnz() 

(6096, 2261895)

In [69]:
# inspect shapes
X_test.shape, X_test_tfidf.shape[0]

((6096,), 6096)

In [70]:
preds = model.predict(X_test_tfidf)
probas = model.predict_proba(X_test_tfidf)

In [71]:
from collections import Counter
Counter(preds)

Counter({8: 6096})

In [72]:
Counter(y_test)

Counter({0: 1174,
         6: 442,
         8: 2814,
         7: 466,
         3: 62,
         2: 208,
         5: 255,
         1: 539,
         4: 136})

In [55]:
Counter(probas)

TypeError: unhashable type: 'numpy.ndarray'

In [58]:
# command to look up
np.vstack((y_test[:20], preds[:20]))

array([[0, 6, 0, 0, 8, 8, 8, 7, 8, 3, 8, 0, 7, 2, 6, 3, 5, 0, 8, 0],
       [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]])

In [59]:
for doc, category in zip(X_test[:20], predicted[:20]):
    print('%r => %s' % (doc[:20], category))

'The Legendary Pink D' => 8
'Can you resist a gir' => 8
'As Generationals,\xa0Ne' => 8
'Few electronic artis' => 8
'"We tried hard to so' => 8
'Nothing about Ezra F' => 8
"Childhood isn't kids" => 8
'One of the quietly b' => 8
'Pairing R&B chops wi' => 8
"If there's one compl" => 8
'Who will finally put' => 8
'Like his contemporar' => 8
'Earlier this year, t' => 8
'For those who think ' => 8
"Last year's self-tit" => 8
'"Headphones on. The ' => 8
"It's probably asking" => 8
'In their half-decade' => 8
'The Society of Rocke' => 8
'As a co-founder of B' => 8


#### Evaluation Metrics for Naive Bayes Classifier

In [60]:
np.mean(predicted == y_test)

0.46161417322834647

In [ ]:
# text_clf = Pipeline([
#    ('vect', CountVectorizer()),
#    ('tfidf', TfidfTransformer()),
#    ('clf', MultinomialNB()),
# ])

In [ ]:
# twenty_test = fetch_20newsgroups(subset='test',
#    categories=categories, shuffle=True, random_state=42)
# docs_test = twenty_test.data
# predicted = text_clf.predict(docs_test)
# np.mean(predicted == twenty_test.target)            


#### linear support vector machine (SVM)

In [85]:
# text_clf = Pipeline([
#   ('vect', CountVectorizer()),
#   ('tfidf', TfidfTransformer()),
#   ('clf', SGDClassifier(loss='hinge', penalty='l2',
#                         alpha=1e-3, random_state=42,
#                         max_iter=5, tol=None)),
# ])

# text_clf.fit(X_train, y_train)  

# predicted_svm = text_clf.predict(X_test)
# np.mean(predicted_svm == y_test)          

In [86]:
# tokenize and vectorize per sklearn workflow
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

In [88]:
model_svm = SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)

model_svm.fit(X_train_tf, y_train)

preds_svm = model_svm.predict(X_test_tfidf)
# probas_svm = model_svm.predict_proba(X_test_tfidf) # TODO (Lee) - issue with probas

In [89]:
np.mean(preds_svm == y_test)

0.5611876640419947

#### McNemar test

Instance,	Classifier1 Correct,	Classifier2 Correct
1			Yes						No
2			No						No
3			No						Yes
4			No						No
5			Yes						Yes
6			Yes						Yes
7			Yes						Yes
8			No						No
9			Yes						No
10			Yes						Yes

In [77]:
len(predicted), len(y_test)

(6096, 6096)

In [ ]:
np.vstack((y_test[:20], preds[:20]))

In [191]:
cont_table = np.vstack((y_test, preds, preds_svm)).T
cont_table[:5]

array([[0, 8, 8],
       [6, 8, 8],
       [0, 8, 8],
       [0, 8, 0],
       [8, 8, 8]])

In [198]:
cont_table.shape

(6096, 3)

In [ ]:
cont_table

In [216]:
cont_data = {'label': list(cont_table[:, 0]), 
        'nb_pred': list(cont_table[:,1]), 
        'svm_pred': list(cont_table[:, 2])}

In [217]:
cont_data

{'label': [0,
  6,
  0,
  0,
  8,
  8,
  8,
  7,
  8,
  3,
  8,
  0,
  7,
  2,
  6,
  3,
  5,
  0,
  8,
  0,
  0,
  0,
  0,
  7,
  1,
  8,
  8,
  8,
  0,
  1,
  8,
  0,
  8,
  8,
  0,
  8,
  7,
  0,
  8,
  8,
  7,
  7,
  7,
  8,
  0,
  8,
  0,
  8,
  1,
  0,
  8,
  0,
  8,
  8,
  6,
  8,
  7,
  2,
  8,
  8,
  4,
  8,
  8,
  5,
  5,
  0,
  0,
  4,
  8,
  0,
  1,
  4,
  0,
  0,
  8,
  7,
  8,
  8,
  8,
  0,
  8,
  8,
  0,
  8,
  0,
  1,
  8,
  8,
  7,
  8,
  0,
  0,
  0,
  6,
  8,
  0,
  8,
  5,
  0,
  4,
  8,
  1,
  8,
  8,
  1,
  8,
  4,
  6,
  8,
  0,
  1,
  8,
  0,
  0,
  2,
  8,
  6,
  8,
  8,
  8,
  8,
  6,
  0,
  8,
  8,
  8,
  8,
  0,
  8,
  0,
  0,
  0,
  7,
  8,
  8,
  7,
  0,
  8,
  8,
  3,
  8,
  7,
  8,
  8,
  8,
  6,
  1,
  7,
  3,
  1,
  1,
  0,
  8,
  7,
  8,
  8,
  8,
  2,
  7,
  7,
  0,
  6,
  1,
  6,
  7,
  6,
  8,
  0,
  2,
  8,
  8,
  6,
  7,
  8,
  8,
  2,
  1,
  8,
  8,
  8,
  8,
  8,
  2,
  7,
  7,
  8,
  8,
  5,
  8,
  8,
  6,
  0,
  0,
  1,
  0,
  2,
  6,
  0,
 

In [199]:
counter = 0
for i in cont_table:
    for j in i:
        if j == j == j:
            counter +=1
print(counter)
        

18288


In [207]:
df_label = pd.DataFrame(cont_table)
df_label.columns = ['label', 'nb_pred', 'svm_pred']
df_label.head(5)

,label,nb_pred,svm_pred
0,0,8,8
1,6,8,8
2,0,8,8
3,0,8,0
4,8,8,8


In [ ]:
df_test['yes_no']

In [201]:
df_test = pd.DataFrame()

""


statistic = (Yes/No - No/Yes)^2 / (Yes/No + No/Yes)

statistic = (Yes/No - No/Yes)^2 / (Yes/No + No/Yes)
Where Yes/No is the count of test instances that Classifier1 got correct and Classifier2 got incorrect, and No/Yes is the count of test instances that Classifier1 got incorrect and Classifier2 got correct.

In [146]:
def yes_no(df):
    yes_no = []
    for i, j in df.iteritems:
        if df[0] == df[1] and i[0] != df[2]:
            yes_no.append()
        return yes_no
    for key, value in df.iteritems(): 
    print(key, value) 
    print() 

In [165]:
for idx, row in df_conts.iterrows():
    yes_no = []
    counter = 0
    if row['target'] == row['nb_pred']:
        counter +=1   

0
0
0
0
1
1
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
1
0
0
1
0
1
1
0
1
0
0
1
1
0
0
0
1
0
1
0
1
0
0
1
0
1
1
0
1
0
0
1
1
0
1
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
1
1
0
1
1
0
1
0
0
1
1
0
1
0
0
0
0
1
0
1
0
0
0
1
0
1
1
0
1
0
0
1
0
0
1
0
0
0
1
0
1
1
1
1
0
0
1
1
1
1
0
1
0
0
0
0
1
1
0
0
1
1
0
1
0
1
1
1
0
0
0
0
0
0
0
1
0
1
1
1
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
1
1
0
0
1
1
1
1
1
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
0
0
0
1
0
0
1
0
1
1
0
1
1
1
0
0
0
1
0
0
0
0
0
1
1
1
1
0
1
0
0
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
0
0
0
0
1
0
1
0
1
0
0
1
0
1
0
1
1
1
0
0
1
0
0
0
0
0
1
1
1
1
0
1
0
0
0
0
1
0
1
0
0
1
0
0
1
1
1
1
0
1
1
0
1
0
0
0
0
0
1
0
0
1
1
0
1
1
1
0
0
0
0
1
0
1
0
1
0
1
0
1
0
0
0
0
0
1
0
1
0
1
1
1
0
1
1
0
1
0
0
1
0
0
1
0
0
0
1
1
0
1
1
1
0
0
0
1
0
1
0
0
1
0
1
1
0
0
1
0
0
0
0
1
0
0
1
1
0
0
1
0
0
1
0
0
1
0
0
1
1
1
1
1
1
0
1
1
0
0
1
1
1
0
1
1
0
1
1
0
0
1
1
0
0
1
1
1
0
0
0
1
0
0
1
0
1
0
1
1
1
0
1
1
1
0
1
0
1
0
1
0
1
0
1
1
0
1
0
0
1
1
1
1
0
1
0
0
0
0
0
0
1
1
1
0
0
1
1
0
0
1
1
1
1
1
0
0
1
1
0
0
0


0
0
0
1
0
0
0
1
0
0
1
1
0
0
1
0
0
1
1
0
0
0
1
0
0
1
1
1
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
0
0
0
1
0
0
0
0
1
1
1
0
0
0
0
1
1
0
0
0
1
0
1
0
0
1
1
0
1
1
1
1
1
0
0
1
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
1
1
1
1
0
0
0
0
1
1
0
1
0
1
1
1
0
1
1
1
1
0
1
0
0
1
0
0
1
0
0
1
1
1
0
1
0
1
0
1
1
0
0
1
0
0
1
0
1
1
1
1
1
0
1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
1
1
0
1
0
1
1
0
1
1
0
0
1
1
0
1
0
1
0
0
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
0
0
1
1
0
1
0
1
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
0
1
1
0
0
0
0
0
1
1
0
0
0
0
0
1
0
0
0
1
0
1
1
0
1
1
0
0
1
0
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
1
1
1
1
1
0
0
1
1
0
0
1
1
1
0
0
0
1
1
0
0
1
1
0
0
0
0
0
0
1
0
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
1
0
1
0
0
1
1
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
1
0
0
1
1
1
1
0
1
1
0
0
1
0
1
1
0
0
0
0
0
0
0
1
0
0
0
1
0
1
1
0
1
0
0
1
1
1
1
1
0
0
1
0
0
0
1
1
0
1
0
1
1
0
0
1
0
1
1
1
0
1
1
1
1
0
0
0
0
1
0
1
0
0
1
1
0
0
1
0
1
0
0
0
1
1
0
1
1
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
0
1
1
0
0
1
1
1
1
0
0
1
1
1
0
0
1
0
0
0
0
1
1
1
0
0
0
0
0
1
1
0
1
0
1
1
0


In [ ]:
sum = 0
    for x in l:
        sum += x
    return sum

In [113]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 6096 entries, 0 to 6095
dtypes: int64(6096)
memory usage: 143.0 KB


In [114]:
df_label

,0,1,2,3,4,5,6,7,8,9,...,6086,6087,6088,6089,6090,6091,6092,6093,6094,6095
0,0,6,0,0,8,8,8,7,8,3,...,7,8,8,3,8,1,8,0,0,0
1,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
2,8,8,8,0,8,8,8,7,8,8,...,7,8,8,8,0,8,8,8,8,7


In [ ]:
# define contingency table
table = [[4, 2],
		 [1, 3]]


1
2
3
						Classifier2 Correct,	Classifier2 Incorrect
Classifier1 Correct 	??						??
Classifier1 Incorrect 	?? 						??
In the case of the first cell in the table, we must sum the total number of test instances that Classifier1 got correct and Classifier2 got correct. For example, the first instance that both classifiers predicted correctly was instance number 5. The total number of instances that both classifiers predicted correctly was 4.

Another more programmatic way to think about this is to sum each combination of Yes/No in the results table above.

						Classifier2 Correct,	Classifier2 Incorrect
Classifier1 Correct 	Yes/Yes					Yes/No
Classifier1 Incorrect 	No/Yes 					No/No
1
2
3
						Classifier2 Correct,	Classifier2 Incorrect
Classifier1 Correct 	Yes/Yes					Yes/No
Classifier1 Incorrect 	No/Yes 					No/No
The results organized into a contingency table are as follows:

						Classifier2 Correct,	Classifier2 Incorrect
Classifier1 Correct 	4						2
Classifier1 Incorrect 	1 						3
1
2
3
						Classifier2 Correct,	Classifier2 Incorrect
Classifier1 Correct 	4						2
Classifier1 Incorrect 	1 						3


In [ ]:
# calculate mcnemar test
result = mcnemar(table, exact=True)

# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))

# interpret the p-value
alpha = 0.05

if result.pvalue > alpha:
    print('Same proportions of errors (fail to reject H0)')
else:
    print('Different proportions of errors (reject H0)')